In [1]:
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
from torchvision import models, transforms
from pathlib import Path
from torchvision.datasets import ImageFolder


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load classes
data_split_dir = Path("../data/splits")
train_dataset = ImageFolder(root=data_split_dir / "train")
classes = train_dataset.classes


# Load model
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512, len(classes))
)
model.load_state_dict(torch.load("../models/best_resnet50.pth", map_location=device))
model = model.to(device)
model.eval()

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

uploaded_image_path = None


d:\Software\Anaconda\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Software\Anaconda\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\coder\AppData\Local\Temp\ipykernel_18968\3464327493.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `w

In [2]:

# --------------------------
# 2. Prediction Function
# --------------------------
def predict_leaf(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        conf, pred = torch.max(probs, 1)
    return pred.item(), conf.item()


def upload_image():
    global uploaded_image_path
    file_path = filedialog.askopenfilename(filetypes=[("Image Files","*.jpg *.jpeg *.png")])
    if file_path:
        uploaded_image_path = file_path
        img = Image.open(file_path).convert("RGB")
        img = img.resize((300,300))
        img_tk = ImageTk.PhotoImage(img)
        img_display.configure(image=img_tk)
        img_display.image = img_tk
        result_label.configure(text="")
        conf_bar.set(0)
        info_label.configure(text="")

def check_status():
    if not uploaded_image_path:
        ctk.CTkMessagebox(title="Warning", message="Please upload a leaf image first!")
        return

    # Predict class and confidence
    pred_idx, conf = predict_leaf(uploaded_image_path)

    # Clean class name: replace underscores and capitalize words
    disease_name = classes[pred_idx].replace("_", " ").title()

    # Update result label
    result_label.configure(text=f"Disease: {disease_name}", font=ctk.CTkFont(size=16, weight="bold"))

    # Update confidence bar
    conf_bar.set(conf*100)

 


In [5]:
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
from torchvision import models, transforms
from pathlib import Path
from torchvision.datasets import ImageFolder

# --------------------------
# 1. Device & Model Setup
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load classes
data_split_dir = Path("../data/splits")
train_dataset = ImageFolder(root=data_split_dir / "train")
classes = train_dataset.classes

# Load model
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512, len(classes))
)
model.load_state_dict(torch.load("../models/best_resnet50.pth", map_location=device))
model = model.to(device)
model.eval()

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

uploaded_image_path = None

# --------------------------
# 2. Prediction Function
# --------------------------
def predict_leaf(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        conf, pred = torch.max(probs, 1)
    return pred.item(), conf.item()

# --------------------------
# 3. GUI Functions
# --------------------------
def upload_image():
    global uploaded_image_path
    file_path = filedialog.askopenfilename(filetypes=[("Image Files","*.jpg *.jpeg *.png")])
    if file_path:
        uploaded_image_path = file_path
        img = Image.open(file_path).convert("RGB")
        img = img.resize((300,300))
        img_tk = ImageTk.PhotoImage(img)
        img_display.configure(image=img_tk)
        img_display.image = img_tk
        result_label.configure(text="")
        conf_bar.set(0)

def check_status():
    if not uploaded_image_path:
        ctk.CTkMessagebox(title="Warning", message="Please upload a leaf image first!")
        return

    # Predict class and confidence
    pred_idx, conf = predict_leaf(uploaded_image_path)

    # Replace underscores with spaces and capitalize
    disease_name = classes[pred_idx].replace("_", " ").title()

    # Update result label
    result_label.configure(text=f"Disease: {disease_name}", font=ctk.CTkFont(size=18, weight="bold"))

    # Update confidence bar
    conf_bar.set(conf*100)

# --------------------------
# 4. Build Modern GUI
# --------------------------
ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("Plant Disease Detector")
app.geometry("800x500")
app.resizable(False, False)

# Title
title_label = ctk.CTkLabel(app, text="🌿 Plant Disease Detector 🌿", font=ctk.CTkFont(size=22, weight="bold"))
title_label.pack(pady=10)

# Main split frame
main_frame = ctk.CTkFrame(app)
main_frame.pack(fill="both", expand=True, padx=20, pady=10)

# Left frame (buttons + confidence bar)
left_frame = ctk.CTkFrame(main_frame)
left_frame.pack(side="left", fill="y", padx=10, pady=10)

upload_btn = ctk.CTkButton(left_frame, text="Upload Leaf Image", command=upload_image, width=180)
upload_btn.pack(pady=20)

check_btn = ctk.CTkButton(left_frame, text="Check Status", command=check_status, width=180)
check_btn.pack(pady=20)

# Confidence bar
conf_bar_label = ctk.CTkLabel(left_frame, text="Confidence:")
conf_bar_label.pack(pady=(20,5))
conf_bar = ctk.CTkProgressBar(left_frame, width=180)
conf_bar.set(0)
conf_bar.pack(pady=5)

# Right frame (image + result)
right_frame = ctk.CTkFrame(main_frame)
right_frame.pack(side="right", fill="both", expand=True, padx=10, pady=10)

# Image display
img_display = ctk.CTkLabel(right_frame, text="")
img_display.pack(pady=10)

# Prediction result
result_label = ctk.CTkLabel(right_frame, text="", font=ctk.CTkFont(size=18, weight="bold"))
result_label.pack(pady=20)

# Run app
app.mainloop()


C:\Users\coder\AppData\Local\Temp\ipykernel_18968\1585116944.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/best_resnet50.p